# CLOB Risk Model - Executive Summary

## Professional Risk Analysis Dashboard

This notebook provides a comprehensive risk analysis for perpetual CLOBs, combining CEX and AMM liquidity metrics with advanced visualizations.

In [ ]:
# Setup and imports
import sys
import os
sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from risk_model import (
    config, binance_data, uniswap_data, 
    pricing, plotting, metrics, scenarios,
    volume_analysis
)

# Professional styling
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', '{:.2f}'.format)

print("Risk Analysis Framework v0.1.0")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("="*50)

## 1. Market Overview & Data Collection

In [ ]:
# Load configuration and fetch market data
markets_config = config.load_markets()
risk_params = config.get_risk_parameters()

# Initialize data fetchers
binance = binance_data.BinanceDataFetcher()

# Fetch data for all configured markets
market_data = {}
print("Fetching market data...\n")

for market in markets_config['markets'][:3]:  # Top 3 markets
    symbol = market['binance_symbol_perp']
    print(f"📊 {market['name']} ({symbol})")
    
    try:
        # Fetch all data
        orderbook = binance.get_orderbook(symbol, limit=500)
        ticker = binance.get_ticker_24h(symbol)
        klines = binance.get_klines(symbol, interval='1h', lookback_days=30)
        
        # Process liquidity
        liq_curve = pricing.compute_orderbook_liquidity_curve(orderbook)
        cex_depth = pricing.depth_at_impact(liq_curve, risk_params['max_price_impact_pct'])
        
        # Calculate volatility
        vol = pricing.realized_vol(klines)
        
        # Volume analysis
        vol_analysis, vol_profile = volume_analysis.analyze_volume_patterns(klines)
        
        market_data[market['name']] = {
            'symbol': symbol,
            'orderbook': orderbook,
            'liquidity_curve': liq_curve,
            'ticker': ticker,
            'klines': klines,
            'volume_analysis': vol_analysis,
            'current_price': klines['close'].iloc[-1],
            'volume_24h_usd': float(ticker['quoteVolume']),
            'cex_liquidity_usd': cex_depth,
            'total_liquidity_usd': cex_depth,  # CEX only for now
            'realized_vol': vol,
            'asset': market['name'].split('-')[0],
            'oracle_meta': {'type': market['oracle']}
        }
        print(f"  ✓ Price: ${market_data[market['name']]['current_price']:,.2f}")
        print(f"  ✓ 24h Volume: ${market_data[market['name']]['volume_24h_usd']:,.0f}")
        print(f"  ✓ Liquidity @ 1%: ${market_data[market['name']]['cex_liquidity_usd']:,.0f}")
        print(f"  ✓ Volatility: {market_data[market['name']]['realized_vol']*100:.1f}%\n")
        
    except Exception as e:
        print(f"  ✗ Error: {e}\n")
        continue

## 2. Professional Visualizations

### 2.1 Market Depth Analysis

In [ ]:
# Create professional depth visualization
if market_data:
    first_market = list(market_data.keys())[0]
    liq_curve = market_data[first_market]['liquidity_curve']
    
    fig, axes = plotting.plot_orderbook_depth(liq_curve, title=f"{first_market} Order Book Depth")
    plt.show()

### 2.2 Risk Score Heatmap

In [ ]:
# Calculate risk scores for all markets
risk_scores = []
market_names = []

for name, data in market_data.items():
    # Calculate all metrics
    all_metrics = metrics.calculate_all_metrics(data)
    
    scores = all_metrics['scores']
    risk_scores.append([
        scores['liquidity'],
        scores['volatility'],
        scores['oracle'],
        scores['composite']
    ])
    market_names.append(name)

# Create risk matrix DataFrame
risk_matrix = pd.DataFrame(
    risk_scores, 
    index=market_names,
    columns=['Liquidity', 'Volatility', 'Oracle', 'Composite']
)

# Plot professional heatmap
fig, ax = plotting.plot_risk_heatmap(risk_matrix, "Risk Assessment Matrix")
plt.show()

print("\nRisk Score Summary:")
print(risk_matrix)

### 2.3 Volatility Surface

In [ ]:
# Prepare volatility data for multiple assets
vol_data = {}

for name, data in market_data.items():
    klines = data['klines']
    
    # Calculate rolling volatility
    returns = np.log(klines['close'] / klines['close'].shift(1)).dropna()
    rolling_vol = returns.rolling(window=24*7).std() * np.sqrt(365 * 24)
    
    vol_df = pd.DataFrame(index=klines.index)
    vol_df['volatility'] = rolling_vol
    
    # Add confidence bands (simplified)
    vol_df['vol_lower'] = vol_df['volatility'] * 0.8
    vol_df['vol_upper'] = vol_df['volatility'] * 1.2
    
    vol_data[name] = vol_df.dropna()

# Plot volatility surface
fig, ax = plotting.plot_volatility_surface(vol_data, "Volatility Dynamics Across Markets")
plt.show()

### 2.4 Volume-Depth Correlation Analysis

In [ ]:
# Analyze volume-depth relationship for first market
if market_data:
    first_market = list(market_data.keys())[0]
    data = market_data[first_market]
    
    # Prepare volume and depth time series
    vol_analysis = data['volume_analysis']
    
    # Create depth time series (simulated for demonstration)
    depth_df = pd.DataFrame(index=vol_analysis.index)
    depth_df['depth_1pct'] = data['cex_liquidity_usd'] * (1 + 0.1 * np.random.randn(len(depth_df)))
    depth_df['depth_2pct'] = depth_df['depth_1pct'] * 1.5
    
    volume_data = {
        'volume': vol_analysis[['volume']],
        'depth': depth_df
    }
    
    fig, axes = plotting.plot_volume_depth_correlation(volume_data)
    plt.show()

## 3. Risk Limits & Recommendations

In [ ]:
# Generate OI limits table
limits_data = []

for name, data in market_data.items():
    all_metrics = metrics.calculate_all_metrics(data)
    
    limits_data.append({
        'Market': name,
        'Risk Score': all_metrics['scores']['composite'],
        'Total Liquidity': data['total_liquidity_usd'],
        'Max OI': all_metrics['limits']['max_oi_usd'],
        'Max Position': all_metrics['limits']['max_position_usd'],
        'OI/Volume Ratio': all_metrics['health']['oi_to_volume_ratio'],
        'Health Status': all_metrics['health']['oi_health_status']
    })

limits_df = pd.DataFrame(limits_data)

# Display formatted table
print("\n" + "="*80)
print("RECOMMENDED POSITION LIMITS")
print("="*80 + "\n")

# Format for display
display_df = limits_df.copy()
for col in ['Total Liquidity', 'Max OI', 'Max Position']:
    display_df[col] = display_df[col].apply(lambda x: f'${x/1e6:.1f}M')
display_df['OI/Volume Ratio'] = display_df['OI/Volume Ratio'].apply(lambda x: f'{x:.2f}x')

print(display_df.to_string(index=False))

# Create visual table
fig, ax = plotting.plot_oi_limits_table(display_df)
plt.show()

## 4. Stress Testing Scenarios

In [ ]:
# Run stress scenarios for the first market
if market_data:
    first_market = list(market_data.keys())[0]
    data = market_data[first_market]
    
    # Get recommended max OI
    all_metrics = metrics.calculate_all_metrics(data)
    max_oi = all_metrics['limits']['max_oi_usd']
    
    # Run scenarios
    stress_scenarios = scenarios.run_all_scenarios(data, max_oi)
    
    # Summarize results
    scenario_df = scenarios.summarize_scenarios(stress_scenarios)
    
    # Plot scenario impacts
    fig, ax = plotting.plot_scenario_impacts(scenario_df)
    plt.show()
    
    print("\nStress Test Summary:")
    print(scenario_df[['scenario', 'severity', 'impact_pct']].to_string(index=False))

## 5. Volume Analysis & Market Quality

In [ ]:
# Generate volume analysis reports
volume_reports = []

for name, data in market_data.items():
    # Calculate liquidity scores
    book_ticker = binance.get_book_ticker(data['symbol'])
    spread_metrics = pricing.calculate_spread_metrics(book_ticker)
    
    liq_score = volume_analysis.calculate_liquidity_score(
        data['volume_24h_usd'],
        data['cex_liquidity_usd'],
        spread_metrics['spread_bps']
    )
    
    # Analyze volume-volatility relationship
    vol_vol_analysis = volume_analysis.analyze_volume_volatility_relationship(
        data['volume_analysis'],
        data['realized_vol']
    )
    
    volume_reports.append({
        'Market': name,
        'Liquidity Score': f"{liq_score['composite_liquidity_score']:.3f}",
        'Liquidity Rating': liq_score['liquidity_rating'],
        'Volume Consistency': f"{vol_vol_analysis['volume_consistency']:.3f}",
        'High Volume Days': f"{vol_vol_analysis['pct_high_volume_days']:.1f}%",
        'Vol/Activity Ratio': f"{vol_vol_analysis['vol_activity_ratio']:.2f}"
    })

volume_df = pd.DataFrame(volume_reports)

print("\n" + "="*80)
print("MARKET QUALITY ASSESSMENT")
print("="*80 + "\n")
print(volume_df.to_string(index=False))

## 6. P&L Distribution Analysis

In [ ]:
# Simulate P&L scenarios based on historical data
pnl_scenarios = pd.DataFrame()

for name, data in market_data.items():
    klines = data['klines']
    returns = klines['close'].pct_change().dropna() * 100  # Percentage returns
    
    # Simulate leveraged P&L (5x leverage)
    leverage = 5
    pnl_scenarios[f'{name}_5x'] = returns * leverage

# Plot P&L distribution
if not pnl_scenarios.empty:
    fig, axes = plotting.plot_pnl_distribution(pnl_scenarios, "Leveraged P&L Distribution Analysis")
    plt.show()
    
    # Calculate risk metrics
    print("\nRisk Metrics (5x Leverage):")
    for col in pnl_scenarios.columns:
        var_95 = np.percentile(pnl_scenarios[col].dropna(), 5)
        cvar_95 = pnl_scenarios[col][pnl_scenarios[col] <= var_95].mean()
        max_dd = (pnl_scenarios[col].cumsum()).min()
        
        print(f"\n{col}:")
        print(f"  VaR 95%: {var_95:.2f}%")
        print(f"  CVaR 95%: {cvar_95:.2f}%")
        print(f"  Max Drawdown: {max_dd:.2f}%")

## Executive Summary & Recommendations

### Key Findings

Based on the comprehensive analysis above, here are the key risk management recommendations:

1. **Position Limits**: Implement the recommended OI limits based on available liquidity and risk scores
2. **Risk Monitoring**: Markets with composite risk scores > 3 require enhanced monitoring
3. **Stress Testing**: Regular stress tests should be conducted, especially for cascade scenarios
4. **Liquidity Management**: Maintain OI/Volume ratios below 5x for healthy market functioning

### Next Steps

1. Implement real-time monitoring of these metrics
2. Set up alerts for risk threshold breaches
3. Integrate AMM liquidity data when available
4. Conduct regular parameter recalibration based on market conditions

In [ ]:
# Save analysis timestamp
print(f"\n" + "="*50)
print(f"Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Framework Version: 0.1.0")
print("="*50)